In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [72]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
# Keras imports
from tensorflow.keras import Sequential,Model
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense,MaxPool1D,Dropout,Embedding,Activation,Convolution1D,add,BatchNormalization,Flatten,Reshape,Concatenate,LSTM,GRU
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping,Callback
from tensorflow.keras.optimizers import Adam,SGD,Adamax,RMSprop
from tensorflow.keras import regularizers
from tensorflow.keras.utils import plot_model



In [75]:
news = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/Combined_News_DJIA.csv')

In [76]:
news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

In [ ]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1989 non-null   object
 1   Label   1989 non-null   int64 
 2   Top1    1989 non-null   object
 3   Top2    1989 non-null   object
 4   Top3    1989 non-null   object
 5   Top4    1989 non-null   object
 6   Top5    1989 non-null   object
 7   Top6    1989 non-null   object
 8   Top7    1989 non-null   object
 9   Top8    1989 non-null   object
 10  Top9    1989 non-null   object
 11  Top10   1989 non-null   object
 12  Top11   1989 non-null   object
 13  Top12   1989 non-null   object
 14  Top13   1989 non-null   object
 15  Top14   1989 non-null   object
 16  Top15   1989 non-null   object
 17  Top16   1989 non-null   object
 18  Top17   1989 non-null   object
 19  Top18   1989 non-null   object
 20  Top19   1989 non-null   object
 21  Top20   1989 non-null   object
 22  Top21   1989 non-null   

In [ ]:
news = news.drop(['Top2', 'Top3', 'Top4', 'Top5', 'Top6', 'Top7', 'Top8', 'Top9', 'Top10', 'Top11', 'Top12', 'Top13', 'Top14', 'Top15', 'Top16', 'Top17', 'Top18', 'Top19', 'Top20', 'Top21', 'Top22', 'Top23', 'Top24', 'Top25'], axis=1)
news.head()

,Date,Label,Top1
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...
4,2008-08-14,1,b'All the experts admit that we should legalis...


In [ ]:
news = news.drop(['Label'], axis=1)
news.head()

,Date,Top1
0,2008-08-08,"b""Georgia 'downs two Russian warplanes' as cou..."
1,2008-08-11,b'Why wont America and Nato help us? If they w...
2,2008-08-12,b'Remember that adorable 9-year-old who sang a...
3,2008-08-13,b' U.S. refuses Israel weapons to attack Iran:...
4,2008-08-14,b'All the experts admit that we should legalis...


In [ ]:
cprt = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/CPRT.csv')
mcd = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/MCD.csv')
mnst = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/MNST.csv')
nvda = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/NVDA.csv')
wst = pd.read_csv('/content/drive/MyDrive/assignment_hacktive8/Phase_2/FP/Final_Project/WST.csv')

In [ ]:
cprt = cprt.drop(['High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)
mcd = mcd.drop(['High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)
mnst = mnst.drop(['High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)
nvda = nvda.drop(['High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)
wst = wst.drop(['High', 'Low', 'Close', 'Adj Close', 'Volume'], axis=1)

In [ ]:
cprt.rename(columns = {'Open':'CPRT'}, inplace = True)
mcd.rename(columns = {'Open':'MCD'}, inplace = True)
mnst.rename(columns = {'Open':'MNST'}, inplace = True)
nvda.rename(columns = {'Open':'NVDA'}, inplace = True)
wst.rename(columns = {'Open':'WST'}, inplace = True)

In [ ]:
mcd = mcd.drop(['Date'], axis=1)
mnst = mnst.drop(['Date'], axis=1)
nvda = nvda.drop(['Date'], axis=1)
wst = wst.drop(['Date'], axis=1)

In [ ]:
stock = pd.concat([cprt,mcd,mnst,nvda,wst], join='inner', axis=1)

In [ ]:
stock.head()

,Date,CPRT,MCD,MNST,NVDA,WST
0,2008-08-08,10.8750,62.389999,3.571667,2.8500,23.910000
1,2008-08-11,11.0925,65.040001,3.878333,2.7700,24.650000
2,2008-08-12,11.1950,65.209999,4.496667,2.8300,24.645000
3,2008-08-13,11.0525,64.010002,4.506667,2.9875,24.674999
4,2008-08-14,10.9450,62.650002,4.466667,3.0425,24.700001


In [ ]:
# compare the num of unique dates
print(len(set(stock['Date'])))
# news 
print(len(set(news['Date'])))

1866
1989


In [ ]:
news = news[news['Date'].isin(stock.Date)]

In [ ]:
# compare the num of unique dates
print(len(set(stock['Date'])))
# news 
print(len(set(news['Date'])))

1866
1866


In [ ]:
# Calculate the difference in opening prices betweeb the following and current day
# The  model will try to predict the change in open value based on today's news
stock = stock.set_index('Date')
stock.head()

,CPRT,MCD,MNST,NVDA,WST
Date,,,,,
2008-08-08,10.8750,62.389999,3.571667,2.8500,23.910000
2008-08-11,11.0925,65.040001,3.878333,2.7700,24.650000
2008-08-12,11.1950,65.209999,4.496667,2.8300,24.645000
2008-08-13,11.0525,64.010002,4.506667,2.9875,24.674999
2008-08-14,10.9450,62.650002,4.466667,3.0425,24.700001


In [ ]:
# Target variable = Tomorrow's open price - today's open price
stock =-1 * stock.diff(periods=1)
stock.head()

,CPRT,MCD,MNST,NVDA,WST
Date,,,,,
2008-08-08,NaN,NaN,NaN,NaN,NaN
2008-08-11,-0.2175,-2.650002,-0.306666,0.0800,-0.740000
2008-08-12,-0.1025,-0.169998,-0.618334,-0.0600,0.005000
2008-08-13,0.1425,1.199997,-0.010000,-0.1575,-0.029999
2008-08-14,0.1075,1.360000,0.040000,-0.0550,-0.025002


In [ ]:
stock['Date']=stock.index
stock= stock.reset_index(drop=True)

In [ ]:
stock = stock[stock.CPRT.notnull()]

In [ ]:
stock.head()

,CPRT,MCD,MNST,NVDA,WST,Date
1,-0.2175,-2.650002,-0.306666,0.0800,-0.740000,2008-08-11
2,-0.1025,-0.169998,-0.618334,-0.0600,0.005000,2008-08-12
3,0.1425,1.199997,-0.010000,-0.1575,-0.029999,2008-08-13
4,0.1075,1.360000,0.040000,-0.0550,-0.025002,2008-08-14
5,-0.2675,-1.479995,-0.533333,-0.2650,-0.750000,2008-08-15


In [ ]:
# Define/Intialize the variables
price= []
headlines=[]

# for all the rows in dataframe
for row in stock.iterrows():
    # define a variable to store all headlines for the day
    daily_headlines=[]
    # Spot the date in given row
    date= row[1]['Date']
    # Store the price for the data
    price.append(row[1]['CPRT'])
    price.append(row[1]['MCD'])
    price.append(row[1]['MNST'])
    price.append(row[1]['NVDA'])
    price.append(row[1]['WST'])

    for row_ in news[news.Date==date].iterrows():
        daily_headlines.append(row_[1]['Top1'])
        
    # Apeend the headlines for the date
    headlines.append(daily_headlines)
    # Track progress
    if len(price) % 500 == 0:
        print(len(price))

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000


In [ ]:
print(headlines[:5])
print(price[:5])

[["b'Why wont America and Nato help us? If they wont help us now, why did we help them in Iraq?'"], ["b'Remember that adorable 9-year-old who sang at the opening ceremonies? That was fake, too.'"], ["b' U.S. refuses Israel weapons to attack Iran: report'"], ["b'All the experts admit that we should legalise drugs '"], ['b"Mom of missing gay man: Too bad he\'s not a 21-year-old cheerleader, then they\'d still be looking for him"']]
[-0.21749999999999936, -2.6500020000000006, -0.3066659999999999, 0.08000000000000007, -0.7399999999999984]


## cleaning

In [ ]:
# price for top 3 headlines
price[:3]

[-0.21749999999999936, -2.6500020000000006, -0.3066659999999999]

In [ ]:
# Normalize opening prices (target values)
max_price= max(price)
min_price= min(price)
mean_price= np.mean(price)
def normalize(price):
    return ((price - min_price)/(max_price-min_price))

In [ ]:
norm_price=[]
for p in price:
    norm_price.append(normalize(p))

In [ ]:
# Check that normalization worked well
print(min(norm_price))
print(max(norm_price))
print(np.mean(norm_price))

0.0
1.0
0.4744069065716782


In [ ]:
# Compare the number of headlines for each day
print(max(len(i) for i in headlines))
print(min(len(i) for i in headlines))
print(np.mean([len(i) for i in headlines]))

1
1
1.0


In [ ]:
norm_price[:3]

[0.46082732374825924, 0.29724273612734886, 0.4548309532047587]

In [ ]:
# remove contractions
def decontracted(phrase):
    if "'" in phrase:
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
    return phrase
text = "I should've studied for the exam so my grades wouldn't be so bad"
text1 = "And now I'm regretting a lot"
print(decontracted(text))
print(decontracted(text1))


I should have studied for the exam so my grades would not be so bad
And now I am regretting a lot


In [ ]:
import tqdm

In [ ]:
def clean_text(text):
    '''Remove unwanted characters and format the text to create fewer nulls word embeddings'''
    
    # convert words into lower case
    text= text.lower()
    
    # Replace contraction with their longest forms
    if True:
        text = text.split()
        new_text = []
        # Remove the contractions
        for word in text:
            new_text.append(decontracted(word))
        # Recreate the sentence
        text= ' '.join(new_text)
        
    # Format words and remove unwanted characters
    text = re.sub(r'&amp;','', text)
    text= re.sub(r'0,0','00', text)
    text = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'\$', ' $ ', text)
    text = re.sub(r'u s ', ' united states ', text)
    text = re.sub(r'u n ', ' united nations ', text)
    text = re.sub(r'u k ', ' united kingdom ', text)
    text = re.sub(r'j k ', ' jk ', text)
    text = re.sub(r' s ', ' ', text)
    text = re.sub(r' yr ', ' year ', text)
    text = re.sub(r' l g b t ', ' lgbt ', text)
    text = re.sub(r'0km ', '0 km ', text)
    text = re.sub(r'b ','', text)

    # Remove stopwords
    text=text.split()
    stops= set(stopwords.words('english'))
    text = [w for w in text if w not in stops]
    text= " ".join(text)
    
    return text

In [ ]:
# Clean the headlines
clean_headlines=[]
for daily_headlines in headlines:
    clean_daily_headlines=[]
    for headline in daily_headlines:
        clean_daily_headlines.append(clean_text(headline))
    clean_headlines.append(clean_daily_headlines)

In [ ]:
clean_headlines[:3]

[['wont america nato help us wont help us help iraq'],
 ['aremember adorable 9 year old sang opening ceremonies fake'],
 ['united states refuses israel weapons attack iran report']]

In [ ]:
print('The numbers we have: {}'.format(len({word: None for headlines in clean_headlines for headline in headlines for word in headline.split()})))

The numbers we have: 7557


In [ ]:
# Create the word vocab 
import collections
words = [word for headlines in clean_headlines for headline in headlines for word in headline.split()]
word_counts = collections.Counter(words)

In [ ]:
word_counts

Counter({'wont': 2,
         'america': 15,
         'nato': 5,
         'help': 22,
         'us': 112,
         'iraq': 25,
         'aremember': 2,
         'adorable': 1,
         '9': 16,
         'year': 84,
         'old': 54,
         'sang': 1,
         'opening': 3,
         'ceremonies': 1,
         'fake': 4,
         'united': 76,
         'states': 59,
         'refuses': 5,
         'israel': 57,
         'weapons': 19,
         'attack': 23,
         'iran': 30,
         'report': 24,
         'experts': 5,
         'admit': 1,
         'legalise': 4,
         'drugs': 33,
         'mom': 1,
         'missing': 4,
         'gay': 12,
         'man': 39,
         'bad': 8,
         '21': 2,
         'cheerleader': 1,
         'would': 46,
         'still': 10,
         'looking': 3,
         'afghan': 12,
         'prison': 20,
         'majority': 3,
         'female': 9,
         'prisoners': 4,
         'serving': 5,
         '20': 17,
         'sentences': 6,
       

In [ ]:
clean_headlines

[['wont america nato help us wont help us help iraq'],
 ['aremember adorable 9 year old sang opening ceremonies fake'],
 ['united states refuses israel weapons attack iran report'],
 ['experts admit legalise drugs'],
 ['mom missing gay man bad 21 year old cheerleader would still looking'],
 ['afghan prison majority female prisoners serving 20 year sentences victims rape'],
 ['man arrested locked five hours taking photo police van ignoring entry sign'],
 ['notwo elderly chinese women sentenced year education labor sought permit demonstrate official olympic protest area'],
 ['british resident held guantanamo bay wins legal battle force foreign office reveal notorture evidence'],
 ['isyria says ready put russian missile system soil counterweight united states plans deploy missile shield poland czech republic'],
 ['n korea kim died 2003 replaced lookalike says waseda professor'],
 ['north korea halts denuclearisation us fails remove list states sponsoring terrorism'],
 ['photos 15 year old

In [ ]:
df = pd.DataFrame(clean_headlines, columns = ['News'])

In [ ]:
df.head()

,News
0,wont america nato help us wont help us help iraq
1,aremember adorable 9 year old sang opening cer...
2,united states refuses israel weapons attack ir...
3,experts admit legalise drugs
4,mom missing gay man bad 21 year old cheerleade...


In [ ]:
stocks= stock.reset_index(drop=True)

In [ ]:
date = stocks.Date

In [ ]:
stocks.head()

,CPRT,MCD,MNST,NVDA,WST,Date
0,-0.2175,-2.650002,-0.306666,0.0800,-0.740000,2008-08-11
1,-0.1025,-0.169998,-0.618334,-0.0600,0.005000,2008-08-12
2,0.1425,1.199997,-0.010000,-0.1575,-0.029999,2008-08-13
3,0.1075,1.360000,0.040000,-0.0550,-0.025002,2008-08-14
4,-0.2675,-1.479995,-0.533333,-0.2650,-0.750000,2008-08-15


In [ ]:
stocks = stocks.drop(['Date'], axis=1)

In [ ]:
stocks.head()

,CPRT,MCD,MNST,NVDA,WST
0,-0.2175,-2.650002,-0.306666,0.0800,-0.740000
1,-0.1025,-0.169998,-0.618334,-0.0600,0.005000
2,0.1425,1.199997,-0.010000,-0.1575,-0.029999
3,0.1075,1.360000,0.040000,-0.0550,-0.025002
4,-0.2675,-1.479995,-0.533333,-0.2650,-0.750000


In [ ]:
# Feature Scaling 

from sklearn.preprocessing import StandardScaler
#default rangenya 0-1. Setiap numerik bakal di convert rangenya jadi 0-1

scaler = StandardScaler()
stockd = scaler.fit_transform(stocks)


In [ ]:
stockd

array([[-1.06181409, -2.82096784, -0.56479323,  0.80776531, -1.37721207],
       [-0.4894599 , -0.15092226, -1.18712005, -0.55788437,  0.04242802],
       [ 0.72990337,  1.32405484,  0.02757816, -1.50896182, -0.02426457],
       ...,
       [ 1.24005013,  1.722411  ,  2.00437252,  2.5879872 ,  4.85396369],
       [-0.80051699, -0.14016242,  1.49852282, -1.65528142,  3.23423819],
       [ 1.68796954, -0.60310452,  0.31378403,  1.56374995,  2.87217471]])

In [ ]:
news = df['News'].values.tolist()
price = stocks
# build train and test datasets

from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test = train_test_split(news, price, test_size=0.2, random_state=4)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
# Split train and val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train , test_size=0.1, random_state=22)

X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

In [ ]:
# Check the lengths
print(len(X_train))
print(len(X_test))
print(len(X_val))

1342
373
150


In [ ]:
X_train

array(['man arrested locked five hours taking photo police van ignoring entry sign',
       'new report world leaders call ending criminalization drug use possession responsible legal regulation psychoactive substances',
       'police arrest man suspected ratko mladi', ...,
       'aman threatened arrest australian anti terrorism act videoing police performing search public',
       'norway money knows biggest challenge oil wealth huge run risk wasting substandard projects profitable enough',
       'istop bill c 15 say mandatory jail terms marijuana'],
      dtype='<U247')

In [ ]:
y_train

array([[ 0.23    ,  0.889999,  0.098333,  0.025   ,  0.915001],
       [ 0.100001,  0.200005, -0.203333,  0.0625  ,  0.02    ],
       [-0.0375  , -0.300003, -0.158333, -0.0775  ,  0.015001],
       ...,
       [ 0.0025  ,  1.459999, -0.071667, -0.02    ,  0.385   ],
       [-0.099998, -0.349998,  0.456667, -0.0125  , -0.110001],
       [ 0.2     ,  0.739997, -0.228333, -0.0625  ,  0.205   ]])

In [ ]:
# Feature Scaling 

from sklearn.preprocessing import StandardScaler
#default rangenya 0-1. Setiap numerik bakal di convert rangenya jadi 0-1

scaler = StandardScaler()
y_trains = scaler.fit_transform(y_train)
y_tests = scaler.transform(y_test)


In [ ]:
y_trains

array([[ 1.20068423,  1.02019454,  0.25783575,  0.266728  ,  1.81956529],
       [ 0.53704388,  0.2449832 , -0.31785508,  0.63679036,  0.08247993],
       [-0.16489392, -0.31677801, -0.23197836, -0.74477577,  0.07277749],
       ...,
       [ 0.03930469,  1.66059208, -0.06658742, -0.17734682,  0.79089942],
       [-0.48394402, -0.37294762,  0.94167017, -0.10333435, -0.16983581],
       [ 1.04753528,  0.85166662, -0.36556437, -0.59675083,  0.44154186]])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# create TF-IDF features
XtrainTF = tfidf_vectorizer.fit_transform(X_train)
XvalTF = tfidf_vectorizer.transform(X_val)
XtestTF = tfidf_vectorizer.transform(X_test)

In [ ]:
XtestTF

<373x6189 sparse matrix of type '<class 'numpy.float64'>'
	with 3637 stored elements in Compressed Sparse Row format>

## Yang gue edit ini Zal

In [ ]:
from sklearn.linear_model import LinearRegression

model_lin_reg = LinearRegression()

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
model1 = MultiOutputRegressor(model_lin_reg)

In [ ]:
model1.fit(XtrainTF, y_trains)

MultiOutputRegressor(estimator=LinearRegression())

In [ ]:
# Set up model and predict
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
## Training model and Predict Error on Train-Set and Test-Set
model1.fit(XtrainTF, y_trains)
y_pred_train1 = model1.predict(XtrainTF)
y_pred_test1 = model1.predict(XtestTF)

## Evaluation of Errors
print('========= MEAN SQUARED ERROR ==========')
print('Error MSE Train-Set :', mean_squared_error(y_train, y_pred_train1))
print('Error MSE Test-Set  :', mean_squared_error(y_test, y_pred_test1))
print('\n')
print('========= MEAN ABSOLUTE ERROR =========')
print('Error MAE Train-Set : ', mean_absolute_error(y_train, y_pred_train1))
print('Error MAE Test-Set  : ', mean_absolute_error(y_test, y_pred_test1))
print('\n')
print('========= R2 SCORE =========')
print('R^2 Score Train-Set : ', r2_score(y_trains, y_pred_train1))
print('R^2 Score Test-Set  : ', r2_score(y_tests, y_pred_test1))

========= MEAN SQUARED ERROR ==========
Error MSE Train-Set : 0.3860132050813719
Error MSE Test-Set  : 0.7687754903757568


========= MEAN ABSOLUTE ERROR =========
Error MAE Train-Set :  0.37616569398224164
Error MAE Test-Set  :  0.6495890084051845


========= R2 SCORE =========
R^2 Score Train-Set :  0.9999999999999885
R^2 Score Test-Set  :  -0.49085716991652256


In [ ]:
y_pred_test1

array([[-0.69923722, -1.3373652 ,  0.39291042,  0.43572176, -0.82288345],
       [ 0.60632801, -0.40072585, -0.15551012,  1.89806408,  1.50779725],
       [ 0.20221782,  0.14762663,  0.39433835, -0.06060892,  0.66336253],
       ...,
       [-0.45004257,  0.11182031,  0.06000352,  0.04386818,  0.42571251],
       [ 1.42473383,  0.87121871,  0.66252629,  0.32963917, -0.85619345],
       [-0.00613318, -0.01967897, -0.01907904,  0.02110854, -0.03991081]])

### Evaluasi model


In [ ]:
alpha_space = np.logspace(-4, 0, 30)   # Checking for alpha from .0001 to 1 and finding the best value for alpha
alpha_space

array([1.00000000e-04, 1.37382380e-04, 1.88739182e-04, 2.59294380e-04,
       3.56224789e-04, 4.89390092e-04, 6.72335754e-04, 9.23670857e-04,
       1.26896100e-03, 1.74332882e-03, 2.39502662e-03, 3.29034456e-03,
       4.52035366e-03, 6.21016942e-03, 8.53167852e-03, 1.17210230e-02,
       1.61026203e-02, 2.21221629e-02, 3.03919538e-02, 4.17531894e-02,
       5.73615251e-02, 7.88046282e-02, 1.08263673e-01, 1.48735211e-01,
       2.04335972e-01, 2.80721620e-01, 3.85662042e-01, 5.29831691e-01,
       7.27895384e-01, 1.00000000e+00])

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
# Using value of alpha as 0.020092 to get best accuracy for Ridge Regression
ridge = Ridge(alpha = 2, normalize = True)
ridge.fit(XtrainTF, y_trains)

train_score = ridge.score(XtrainTF, y_trains)
print(train_score)
test_score = ridge.score(XtestTF, y_tests)
print(test_score)

0.8596789406759644
-0.11880406454721917


In [ ]:
# Set up model and predict
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
## Training model and Predict Error on Train-Set and Test-Set
ridge.fit(XtrainTF, y_trains)
y_pred_train1 = ridge.predict(XtrainTF)
y_pred_test1 = ridge.predict(XtestTF)

## Evaluation of Errors
print('========= MEAN SQUARED ERROR ==========')
print('Error MSE Train-Set :', mean_squared_error(y_train, y_pred_train1))
print('Error MSE Test-Set  :', mean_squared_error(y_test, y_pred_test1))
print('\n')
print('========= MEAN ABSOLUTE ERROR =========')
print('Error MAE Train-Set : ', mean_absolute_error(y_train, y_pred_train1))
print('Error MAE Test-Set  : ', mean_absolute_error(y_test, y_pred_test1))
print('\n')
print('========= R2 SCORE =========')
print('R^2 Score Train-Set : ', r2_score(y_trains, y_pred_train1))
print('R^2 Score Test-Set  : ', r2_score(y_tests, y_pred_test1))

========= MEAN SQUARED ERROR ==========
Error MSE Train-Set : 0.13715371969603493
Error MSE Test-Set  : 0.4224215434938525


========= MEAN ABSOLUTE ERROR =========
Error MAE Train-Set :  0.22643856324077807
Error MAE Test-Set  :  0.43657047131972904


========= R2 SCORE =========
R^2 Score Train-Set :  0.8596789406759644
R^2 Score Test-Set  :  -0.11880406454721917


In [ ]:
# make predictions for validation set
y_pred = ridge.predict(XtestTF)

# classes
classes = ['CPRT', 'MCD', 'MNST', 'NVDA', 'WST']

# predict
y_pred

array([[-4.49674348e-01, -8.06602450e-01,  2.35614687e-01,
         2.77313518e-01, -3.43710953e-01],
       [ 1.65150243e-01, -2.38731684e-01,  2.28602522e-02,
         4.01019955e-01,  4.41597545e-01],
       [ 1.79033200e-01, -2.92420887e-03,  2.96596719e-01,
         2.99823466e-02,  4.29477646e-01],
       ...,
       [-1.17067701e+00,  1.53203624e-01,  1.81900288e-02,
        -1.62138316e-01, -8.54125110e-03],
       [ 3.29257367e-01,  2.84871575e-01,  1.96087790e-01,
         1.52258884e-01, -2.26571601e-01],
       [ 2.64750099e-03, -1.32918644e-02,  9.32921990e-04,
         4.51318300e-03, -5.09334334e-03]])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
news_transformer = Pipeline(steps=[("vectorizer", TfidfVectorizer())])

preprocessor = Pipeline(steps=[('news_transformer', news_transformer),('ridge', Ridge(alpha = 2, normalize = True))])

In [ ]:
preprocessor.fit(X_train, y_train)

Pipeline(steps=[('news_transformer',
                 Pipeline(steps=[('vectorizer', TfidfVectorizer())])),
                ('ridge', Ridge(alpha=2, normalize=True))])

In [ ]:
preprocessor.predict(X_test)

array([[-0.09328499, -0.73598208,  0.08668901,  0.02607268, -0.19958694],
       [ 0.0271516 , -0.23053614, -0.0247958 ,  0.03860838,  0.20502869],
       [ 0.0298711 , -0.02065048,  0.11864402,  0.00100955,  0.19878414],
       ...,
       [-0.23452057,  0.11831455, -0.02724302, -0.01845886, -0.02689697],
       [ 0.05929817,  0.23550853,  0.06597664,  0.01340035, -0.13923306],
       [-0.00468069, -0.02987844, -0.03628587, -0.00157135, -0.0251205 ]])

In [ ]:
import pickle

In [ ]:
#with open("preprocessor.pkl", "wb") as f:
#    pickle.dump(preprocessor, f)